In [1]:
import pandas as pd
import os
import math
import numpy as np
import itertools as it
import random
import json

In [2]:
def lammps_cin(exp_dir, rnds, trial, shape, params):
    lammps_dir = exp_dir+f"/input/{shape}_{params['bondcoeff']}_{params['r']}_{params['s0']}_{params['s1']}_{trial}.lammps"
    output_dir = f"../output/{shape}_{params['bondcoeff']}_{params['r']}_{params['s0']}_{params['s1']}_{trial}"
    log_dir = f"../log/{shape}_{params['bondcoeff']}_{params['r']}_{params['s0']}_{params['s1']}_{trial}.log"
    with open(lammps_dir, "w") as f:
        f.write('# LAMMPS script to create a bacteria swiming through a polymer network\n')
        f.write(f'#log {log_dir}\n\n')
        f.write(f'''
variable s0 equal {params['s0']}
variable s1 equal {params['s1']}
variable k equal {params['bondcoeff']}*${{s0}}
variable r equal {params['r']}
variable s equal ${{r}}/1
variable pc12 equal round((${{r}}+1)/2*2^(1/6)*10000)/10000
variable pc22 equal round(${{r}}*2^(1/6)*10000)/10000

print "${{pc12}} ${{pc22}}"

units nano 
boundary p p p 
atom_style angle

pair_style lj/cut 1.1225
bond_style harmonic
angle_style harmonic

read_data ./stretched_{params['bondcoeff']}_${{s0}}_${{s1}}.data extra/atom/types 1 extra/bond/types 1 extra/angle/types 1
read_data ./{shape}_${{s0}}_${{s1}}.data add append offset 1 1 1 0 0

pair_coeff 1 1 1 1 1.1225
pair_coeff 1 2 1 1 ${{pc12}}
pair_coeff 2 2 0 1 ${{pc22}}

bond_coeff 1 {params['spring_const']} ${{k}}
bond_coeff 2 0 0

angle_coeff 1 5.52 180
angle_coeff 2 0 0

group polymer type 1
group bacteria type 2
group non_bacteria subtract all bacteria

displace_atoms bacteria move {params['x']} {params['y']} {params['z']} units box

variable x0 equal x[18751]
variable y0 equal y[18751]
variable z0 equal z[18751]

variable dix atom (x-${{x0}})*(${{s}}-1)
variable diy atom (y-${{y0}})*(${{s}}-1)
variable diz atom (z-${{z0}})*(${{s}}-1)

displace_atoms bacteria move v_dix v_diy v_diz units box

neighbor 5 bin
neigh_modify every 1 delay 0 check yes

compute bac_chunk bacteria chunk/atom molecule
compute bac_com bacteria com/chunk bac_chunk
compute com_v bacteria vcm/chunk bac_chunk
compute 1 non_bacteria property/local batom1 batom2 btype
compute 2 non_bacteria bond/local dist engpot force

# Access the coordinates of atom 1 and atom 5
variable x1 equal x[18751]
variable y1 equal y[18751]
variable z1 equal z[18751]
variable x5 equal x[18763]
variable y5 equal y[18763]
variable z5 equal z[18763]

# Access simulation box boundaries
variable xlo equal xlo
variable xhi equal xhi
variable ylo equal ylo
variable yhi equal yhi
variable zlo equal zlo
variable zhi equal zhi

# Calculate the box lengths in each dimension
variable lx equal v_xhi-v_xlo
variable ly equal v_yhi-v_ylo
variable lz equal v_zhi-v_zlo

# Calculate the raw vector components between atom 1 and atom 5
variable dx_raw equal v_x5-v_x1
variable dy_raw equal v_y5-v_y1
variable dz_raw equal v_z5-v_z1

# Apply minimum image convention to correct for PBCs
variable dx equal v_dx_raw-v_lx*round(v_dx_raw/v_lx)
variable dy equal v_dy_raw-v_ly*round(v_dy_raw/v_ly)
variable dz equal v_dz_raw-v_lz*round(v_dz_raw/v_lz)

# Calculate the magnitude of the corrected vector
variable mag equal sqrt(v_dx^2+v_dy^2+v_dz^2)

# Calculate force
variable fx equal v_dx/v_mag*0.1
variable fy equal v_dy/v_mag*0.1
variable fz equal v_dz/v_mag*0.1

# Calculate torque
variable tx equal v_dx/v_mag*0.5*${{s}}^2
variable ty equal v_dy/v_mag*0.5*${{s}}^2
variable tz equal v_dz/v_mag*0.5*${{s}}^2

thermo 5000
timestep 0.01

fix allnve all nve/limit 0.1
fix freeze bacteria setforce 0 0 0
fix langevin polymer langevin 300 0 1 456789 zero no

run 5000

unfix allnve
unfix freeze
unfix langevin

fix allnve all nve
fix freeze bacteria setforce 0 0 0
fix langevin polymer langevin 0 0 1 456789 zero no

run 15000

unfix allnve
unfix freeze
unfix langevin

#velocity bacteria create 500.0 {rnds[0]} mom no rot no  # Assign random velocities
#fix allnve all nve
fix nonbac_nve non_bacteria nve # commented off for fixed beads
fix bac_rigid bacteria rigid/nve/small molecule langevin 0 0 1 {rnds[1]} # set bacteria as rigid
fix bac_force bacteria addforce v_fx v_fy v_fz
fix bac_torque bacteria addtorque v_tx v_ty v_tz
fix non_bac_langevin non_bacteria langevin 0 0 1 72601 zero no # commented off for fixed beads
#fix fixedbeads non_bacteria setforce 0.0 0.0 0.0 # for fixed beads
fix com bacteria ave/time {params['timestep']} 1 {params['timestep']} c_bac_com[*] c_com_v[*] file {output_dir}.com mode vector

#dump all all movie {params['timestep']} {output_dir}_all.mpeg type type zoom 1 box yes 0.01 view 120 85 size 1920 1080 shiny 0.5
#dump bac bacteria movie {params['timestep']} {output_dir}_bac.mpeg type type zoom 1 box yes 0.01 view 120 85 size 1920 1080 shiny 0.5
dump 3 bacteria custom {params['timestep']} {output_dir}.atoms id type x y z vx vy vz
dump 4 non_bacteria local 500000 {output_dir}.bond index c_1[1] c_1[2] c_2[1] c_2[2] c_2[3]

#dump_modify all adiam 1 1.0
#dump_modify all adiam 2 ${{r}}
#dump_modify bac adiam 1 1.0
#dump_modify bac adiam 2 ${{r}}

run 500000
''')

        


In [3]:
n = 1000 # n different runs
# Generate three random 5-digit numbers
rnds = [random.randint(10000, 99999) for _ in range(n*2)] # random seeds for polymer generation
# rnds

In [4]:
exp_dir = f'../springconst_0.2/'
exp_dir

'../springconst_0.2/'

In [5]:
# exp_dir = f'../{"_".join(map(str, n_poly))}_compact{num_atoms}_l1'
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    os.makedirs(f'{exp_dir}/input')
    os.makedirs(f'{exp_dir}/output')
    os.makedirs(f'{exp_dir}/log')
    os.makedirs(f'{exp_dir}/slurm_log')
    os.makedirs(f'{exp_dir}/code')
    os.makedirs(f'{exp_dir}/analysis')

In [6]:
params = {'r': 1.0, # radius of bacteria atom 
's0': 0.5,  # shrink factor
's1': 1.2, # stretch factor
'timestep': 500, 
'bondcoeff': 2.5, 
'thermo': 10000, 
'spring_const': 0.2} # dump every timestep

## if using rnds from other files

In [7]:
with open(os.path.join('../1000runs/', 'rnd.json')) as f:
    temp = json.load(f)

In [8]:
rnds = temp['rnd']

In [9]:
xs = temp['x']
ys = temp['y']
zs = temp['z']
for i in range(n):
    rnd = rnds[i*2:(i+1)*2]
    x, y, z = xs[i], ys[i], zs[i]
    # x, y, z = [round(random.uniform(-30, 30), 1) for _ in range(3)]
    params['x'] = x
    params['y'] = y
    params['z'] = z
    # print(params)
    lammps_cin(exp_dir=exp_dir, rnds=rnd, trial=i, shape='str', params=params)
    lammps_cin(exp_dir=exp_dir, rnds=rnd, trial=i, shape='cur', params=params)

## if using rnds generated from this run

In [16]:
xs = []
ys = []
zs = []
for i in range(n):
    rnd = rnds[i*2:(i+1)*2]
    x, y, z = random.sample(range(-30, 30), 3)
    xs.append(x)
    ys.append(y)
    zs.append(z)
    # x, y, z = [round(random.uniform(-30, 30), 1) for _ in range(3)]
    params['x'] = x
    params['y'] = y
    params['z'] = z
    # print(params)
    lammps_cin(exp_dir=exp_dir, rnds=rnd, trial=i, shape='str', params=params)
    lammps_cin(exp_dir=exp_dir, rnds=rnd, trial=i, shape='cur', params=params)

In [17]:
import json
data = {'rnd': rnds,
        'x': xs,
        'y': ys,
        'z': zs}
with open(os.path.join(exp_dir, 'rnd.json'), "w") as f:
    json.dump(data, f)

----

In [22]:
bondcoeffs = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
for bc in bondcoeffs:
    params = {'r': 1.5, # radius of bacteria atom 
    's0': 0.5,  # shrink factor
    's1': 1.2, # stretch factor
    'timestep': 500, 
    'bondcoeff': bc} # dump every timestep
    for i in range(n):
        rnd = rnds[i*2:(i+1)*2]
        lammps_cin(exp_dir=exp_dir, rnds=rnd, trial=i, shape='str', params=params)
        lammps_cin(exp_dir=exp_dir, rnds=rnd, trial=i, shape='cur', params=params)